A more complete notebook can be found [here](https://colab.research.google.com/github/google-deepmind/mujoco/blob/main/python/tutorial.ipynb)

In [1]:
import mujoco
import mujoco_viewer

# scene.xml includes the robot model and a simple environment
# model = mujoco.MjModel.from_xml_path("./unitree_go1/scene.xml")
model = mujoco.MjModel.from_xml_path("../unitree_a1/unitree_a1_torque.xml")

# Contains the state of the model (time: .time, pos: .qpos, vel: .qvel) 
data = mujoco.MjData(model)

In [2]:
# Print out the name of the body accessors
try:
    model.geom()
except KeyError as e:
    print(f"model.geom: {e}")
    
try:
    data.geom()
except KeyError as e:
    print(f"data.geom: {e}")
    
try:
    data.body()
except KeyError as e:
    print(f"data.body: {e}")

# Could use the named accessor to get the data
print(f"Before {data.body('FL_calf').xpos=}")
mujoco.mj_kinematics(model, data) # mj_kinematics fills in the values related to the global positions
print(f"After  {data.body('FL_calf').xpos=}")

# Or mj_name2id to get the index
index = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_BODY, 'FL_calf')

# and directly access using the index
print(f"{len(data.xpos)=} -> {data.xpos[index]=}")

model.geom: "Invalid name ''. Valid names: ['FL_foot', 'FR_foot', 'RL_foot', 'RR_foot', 'floor']"
data.geom: "Invalid name ''. Valid names: ['FL_foot', 'FR_foot', 'RL_foot', 'RR_foot', 'floor']"
data.body: "Invalid name ''. Valid names: ['FL_calf', 'FL_hip', 'FL_thigh', 'FR_calf', 'FR_hip', 'FR_thigh', 'RL_calf', 'RL_hip', 'RL_thigh', 'RR_calf', 'RR_hip', 'RR_thigh', 'trunk', 'world']"
Before data.body('FL_calf').xpos=array([0., 0., 0.])
After  data.body('FL_calf').xpos=array([0.183  , 0.13205, 0.23   ])
len(data.xpos)=14 -> data.xpos[index]=array([0.183  , 0.13205, 0.23   ])


In [3]:
model.geom("FL_foot")

<_MjModelGeomViews
  bodyid: array([7])
  conaffinity: array([1])
  condim: array([6])
  contype: array([1])
  dataid: array([-1])
  friction: array([0.8 , 0.02, 0.01])
  gap: array([0.])
  group: array([1])
  id: 30
  margin: array([0.001])
  matid: array([-1])
  name: 'FL_foot'
  pos: array([ 0. ,  0. , -0.2])
  priority: array([1])
  quat: array([1., 0., 0., 0.])
  rbound: array([0.02])
  rgba: array([0.5, 0.6, 0.7, 0.2], dtype=float32)
  sameframe: array([0], dtype=uint8)
  size: array([0.02, 0.  , 0.  ])
  solimp: array([0.015, 1.   , 0.031, 0.5  , 2.   ])
  solmix: array([1.])
  solref: array([0.02, 1.  ])
  type: array([2])
  user: array([], dtype=float64)
>

In [4]:
print(f"Simulated time: {data.time} sec")
print(f"Default timestep: {model.opt.timestep} sec")
print(f"Degrees of freedom: {model.nv=}")

# Position has 1 extra dimension since orientation is represented with
# quaternions (4-values) while angular velocity has 3 values.
print(f"{len(data.qpos)=} {data.qpos=}")
print(f"{len(data.qvel)=} {data.qvel=}")

Simulated time: 0.0 sec
Default timestep: 0.002 sec
Degrees of freedom: model.nv=18
len(data.qpos)=18 data.qpos=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0.])
len(data.qvel)=18 data.qvel=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0.])


In [8]:
import time
import math
import numpy as np
# Setup viewer
viewer = mujoco_viewer.MujocoViewer(model, data)

# Reset the simulation
mujoco.mj_resetDataKeyframe(model, data, 0)

# data.qvel 18 values. It is matched to the order of the "joint" elements in the XML file.
# The order is as follows:
# 0: trunk_tx, 1: trunk_ty, 2: trunk_tz
# 3: trunk_list (roll), 4: trunk_tilt (pitch, +forward), 5: trunk_rotation (yaw, +left)


# Linear Velocity/Torque: 0=forward, 1=left, 2=up
# Anguar Velocity/Torque: 3=roll right, 4=pitch forward, 5=yaw left (from above)
# 6=FR_hip, 7=FR_thigh, 8=FR_calf
# 9=FL_hip, 10=FL_thigh, 11=FL_calf
# 12=RR_hip, 13=RR_thigh, 14=RR_calf
# 15=RL_hip, 16=RL_thigh, 17=RL_calf

# xpos (14x 3d vectors) and xquat (4) are the global position and orientation of the body (i.e. after forward kinematics)
# List of all data fields: https://github.com/atabakd/MuJoCo-Tutorials/blob/d6b86726ca9f01e682a45c4de7c11761ec326d38/include/mjdata.h#L186

# data.qpos[0] = 3.0
# data.ctrl[:] = np.ones(12) * 1.0

for i in range(1000):
    if viewer.is_alive:
        mujoco.mj_step(model, data)
        index = 0
        # if i == 0:
            # data.qvel[index] = -50.0 # Is relative to the world

        viewer.render()
        print(f"{data.time=:.2f} {data.qpos[index]=:.2f} {data.qvel[index]=:.2f}")
        # time.sleep(0.05)
    else:
        break

viewer.close()

data.time=0.00 data.qpos[index]=-0.00 data.qvel[index]=-0.00
data.time=0.00 data.qpos[index]=-0.00 data.qvel[index]=-0.00
data.time=0.01 data.qpos[index]=-0.00 data.qvel[index]=-0.01
data.time=0.01 data.qpos[index]=-0.00 data.qvel[index]=-0.01
data.time=0.01 data.qpos[index]=-0.00 data.qvel[index]=-0.02
data.time=0.01 data.qpos[index]=-0.00 data.qvel[index]=-0.02
data.time=0.01 data.qpos[index]=-0.00 data.qvel[index]=-0.03
data.time=0.02 data.qpos[index]=-0.00 data.qvel[index]=-0.03
data.time=0.02 data.qpos[index]=-0.00 data.qvel[index]=-0.04
data.time=0.02 data.qpos[index]=-0.00 data.qvel[index]=-0.04
data.time=0.02 data.qpos[index]=-0.00 data.qvel[index]=-0.05
data.time=0.02 data.qpos[index]=-0.00 data.qvel[index]=-0.05
data.time=0.03 data.qpos[index]=-0.00 data.qvel[index]=-0.06
data.time=0.03 data.qpos[index]=-0.00 data.qvel[index]=-0.06
data.time=0.03 data.qpos[index]=-0.00 data.qvel[index]=-0.06
data.time=0.03 data.qpos[index]=-0.00 data.qvel[index]=-0.06
data.time=0.03 data.qpos